# Complex in droplet simulation

In [95]:
%pylab inline
import scipy.constants
import time

Populating the interactive namespace from numpy and matplotlib


## Parameters

In [133]:
# Molar protein concentration
#c_molar = 2.5E-6 # [mol/l]
c_molar = 100E-6
#c_molar = 200E-6 # [mol/l]
# Protein mass
m_P = 53.3E3 # [Da]
# Simulation volume
V = 0.125E-18 # [m^3]
# Correction factor
ksi = 20.
# ksi = 30.
# Droplet radius (will be later rescaled to R_D_ksi)
R_D = 9.E-9

## Constants

In [134]:
# Protein mass density
rho_P = 0.84 / (1E-10)**3 # [Da/m^3]

## Derived parameters

In [167]:
# Hard sphere radius of protein
R_S = (m_P/rho_P * 3./4./pi)**(1/3.)
# Rescaled droplet radius
R_D_ksi = R_D * ksi
# SI protein concentration
c = c_molar * scipy.constants.Avogadro / 0.1**3 # [1/m^3]
# Number of particles in virutual box
N = int(round(V * c))
# Edge length of virtual box
L = V**(1/3.)

## Assign positions

In [172]:
def get_distance_sq(pos1, pos2):
    return ((pos1-pos2)**2).sum()

def get_distances_sq(positions, pos1):
    return ((positions-pos1)**2).sum(1)

def populate_box():
    r = array([rand(3)])
    d_sq_min = (R_S/L)**2
    for i in range(1,N):
        d_sq = 0
        while d_sq < d_sq_min:
            new = rand(3)
            d_sq = get_distances_sq(r,new).min()
        r = list(r)
        r.append(new)
        r = array(r)
    r *= L
    return r

def select_drop(r):
    x = ones(3) * L/2.
    pos = [r_close for r_close in r if get_distance_sq(r_close,x)<=R_D_ksi**2]
    return pos

def make_drop():
    seed()
    r = populate_box()
    pos = select_drop(r)
    print len(pos)
    return len(pos)

import multiprocessing
jobs = []
for i in range(20):
    p = multiprocessing.Process(target=make_drop)
    jobs.append(p)
    p.start()
    time.sleep(0.01)
for j in jobs:
    j.join()

1476
1516
1503
1478
1516
1467
1458
1556
1414
1504
1418
1476
1488
1386
1499
1473
1531
1518
1532
1542
